In [1]:
from collections import defaultdict
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary
import pickle

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Create a data dictionary we can use for reference.

We load `data_dict.csv` to get the meaning and data type of each column, as well as possible value-label pairings.

In [2]:
dd = pd.read_csv("data_dict.csv")
dd = dd.drop(["NOTES", "SOURCE"], axis=1)
# clean up the column names
dd.columns = ["_".join(re.split("-| ", col.lower())) for col in dd.columns]
dd = dd.set_index("variable_name")
# remove all value-label info; we'll deal with that next
data_dict = dd[~pd.isna(dd.index)].drop(["value", "label"], axis=1)

In [3]:
data_dict.loc['UNITID']

name_of_data_element       Unit ID for institution
dev_category                                  root
developer_friendly_name                         id
api_data_type                              integer
Name: UNITID, dtype: object

## Create a mapping of integer factors to their actual meanings

In [4]:
label_dict = defaultdict(dict)
last_col = dd.index[0]
for row in dd.iterrows():
    index = row[0]
    if pd.isna(index):
        index = last_col
    else:
        last_col = index
    if not pd.isna(row[1].value):
        label_dict[index][row[1].value] = row[1].label

In [5]:
label_dict['PREDDEG']

{0.0: 'Not classified',
 1.0: 'Predominantly certificate-degree granting',
 2.0: "Predominantly associate's-degree granting",
 3.0: "Predominantly bachelor's-degree granting",
 4.0: 'Entirely graduate-degree granting'}

## First attept of choosing candidate columns

In [6]:
cols = [
    "UNITID",  # maybe concatenate with year
    "INSTNM",
    "STABBR",
    "ACCREDAGENCY",
    "HCM2",
    "MAIN",
    "NUMBRANCH",
    "PREDDEG",  # filter out 4
    "HIGHDEG",  # filter out 0, 1
    "CONTROL",
    "REGION",  # possibly instead of state
    "LOCALE",  # could backfill
   # "LOCALE2",  # was blank in 2017-18
    "CCBASIC",  # could backfill
    "CCUGPROF",  # could backfill
    "CCSIZSET",  # could backfill
    "HBCU",
    "MENONLY",
    "WOMENONLY",
    "RELAFFIL",  # could bin this
    "ADM_RATE",
    "SATVR25",
    "SATVR75",
    "SATMT25",
    "SATMT75",
    "SATWR25",
    "SATWR75",
    "SATVRMID",
    "SATMTMID",
    "SATWRMID",
    "ACTCM25",
    "ACTCM75",
    "ACTEN25",
    "ACTEN75",
    "ACTMT25",
    "ACTMT75",
    "ACTWR25",
    "ACTWR75",
    "ACTCMMID",
    "ACTENMID",
    "ACTMTMID",
    "ACTWRMID",
    "PCIP01",
    "PCIP03",
    "PCIP04",
    "PCIP05",
    "PCIP09",
    "PCIP10",
    "PCIP11",
    "PCIP12",
    "PCIP13",
    "PCIP14",
    "PCIP15",
    "PCIP16",
    "PCIP19",
    "PCIP22",
    "PCIP23",
    "PCIP24",
    "PCIP25",
    "PCIP26",
    "PCIP27",
    "PCIP29",
    "PCIP30",
    "PCIP31",
    "PCIP38",
    "PCIP39",
    "PCIP40",
    "PCIP41",
    "PCIP42",
    "PCIP43",
    "PCIP44",
    "PCIP45",
    "PCIP46",
    "PCIP47",
    "PCIP48",
    "PCIP49",
    "PCIP50",
    "PCIP51",
    "PCIP52",
    "PCIP54",
    "DISTANCEONLY",
    "UGDS",
    "PPTUG_EF",
    "CURROPER",  # filter
    "COSTT4_A",  # collapse with below
    "COSTT4_P",  # collapse with above
    "NUM4_PUB",  # collapse with below; use as filter
    "NUM4_PRIV",  # collapse with above; use as filter
    "TUITIONFEE_IN",  # use as flag for "in-state is different"?
    "TUITIONFEE_OUT",  # see above
    "TUITIONFEE_PROG",  # see above
    "INEXPFTE",
    "AVGFACSAL",
    "PFTFAC",
    "PCTPELL",
    "C150_4",
    "PCTFLOAN",
    "UG25ABV",
    "COMPL_RPY_1YR_RT",
    "COMPL_RPY_3YR_RT",
    "COMPL_RPY_5YR_RT",
    "COMPL_RPY_7YR_RT",
    "GRAD_DEBT_MDN",  # potential part of target
    "COUNT_NWNE_P6",  # potentially filter? /analysis
    "COUNT_WNE_P6",  # see above
    "MN_EARN_WNE_P6",  # target?
    "MD_EARN_WNE_P6",  # target 2?
    "ICLEVEL",  # potentially redundant
    "PRGMOFR",
]

In [7]:
print(f'{len(cols)} candidate columns')

108 candidate columns


## Make a version of our data dictionary just for the columns we're using

In [8]:
df_cols = pd.DataFrame(cols)
df_cols.columns = ["variable"]
df_cols["category"] = df_cols.variable.apply(lambda x: data_dict.loc[x, "dev_category"])
df_cols["type"] = df_cols.variable.apply(lambda x: data_dict.loc[x, "api_data_type"])
df_cols["friendly_name"] = df_cols.variable.apply(
    lambda x: data_dict.loc[x, "developer_friendly_name"]
)
df_cols["explanation"] = df_cols.variable.apply(
    lambda x: data_dict.loc[x, "name_of_data_element"]
)
df_cols.set_index("variable", inplace=True)
df_cols.head()

,category,type,friendly_name,explanation
variable,,,,
UNITID,root,integer,id,Unit ID for institution
INSTNM,school,autocomplete,name,Institution name
STABBR,school,string,state,State postcode
ACCREDAGENCY,school,string,accreditor,Accreditor for institution
HCM2,school,integer,under_investigation,Schools that are on Heightened Cash Monitoring...


## Ingest Data

In [9]:
dfs = dict()
year = 2001
while year < 2018:
    csv_name = f"data/MERGED{year}_{year-1999:02d}_PP.csv"
    print(f"loading {csv_name}")
    dfs[year] = pd.read_csv(
        csv_name, usecols=cols, na_values=["NU", "PrivacySuppressed"]
    )
    dfs[year].loc[:, "YEAR"] = year
    dfs[year].loc[:, "UNITID"] = (
        dfs[year].loc[:, "UNITID"].apply(lambda x: str(x) + "-" + str(year))
    )
    year += 1

loading data/MERGED2001_02_PP.csv
loading data/MERGED2002_03_PP.csv
loading data/MERGED2003_04_PP.csv
loading data/MERGED2004_05_PP.csv
loading data/MERGED2005_06_PP.csv
loading data/MERGED2006_07_PP.csv
loading data/MERGED2007_08_PP.csv
loading data/MERGED2008_09_PP.csv
loading data/MERGED2009_10_PP.csv
loading data/MERGED2010_11_PP.csv
loading data/MERGED2011_12_PP.csv
loading data/MERGED2012_13_PP.csv
loading data/MERGED2013_14_PP.csv
loading data/MERGED2014_15_PP.csv
loading data/MERGED2015_16_PP.csv
loading data/MERGED2016_17_PP.csv
loading data/MERGED2017_18_PP.csv


## Drop null columns

In [10]:
df = pd.concat(dfs.values()).set_index('UNITID')

In [11]:
old_cols = df.columns
new_cols = df.dropna(axis="columns", how="all").columns
#df = df[new_cols].set_index("UNITID")
df_cols = df_cols.loc[df.columns]
label_dict_old = label_dict
label_dict = {
    key: label_dict_old[key]
    for key in (set(df.columns).intersection(set(label_dict_old.keys())))
}
print(f"{len(list(set(old_cols) - set(new_cols)))} columns dropped")

0 columns dropped


/Users/danturkel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  after removing the cwd from sys.path.


In [12]:
top_earners = df.sort_values("MN_EARN_WNE_P6", ascending=False)[
    ["INSTNM", "MN_EARN_WNE_P6"]
].dropna()
top_earners

,INSTNM,MN_EARN_WNE_P6
UNITID,,
215123-2014,Philadelphia College of Osteopathic Medicine,151500.0
239169-2011,Medical College of Wisconsin,151000.0
207315-2009,Oklahoma State University Center for Health Sc...,142800.0
239169-2012,Medical College of Wisconsin,138400.0
239169-2013,Medical College of Wisconsin,128100.0
...,...,...
242954-2007,Modern Hairstyling Institute-Arecibo,11000.0
158282-2005,Bastrop Beauty School Inc,11000.0
241906-2012,Emma's Beauty Academy-Mayaguez,10700.0


In [13]:
df[df['INSTNM'] == 'Hofstra University']['MD_EARN_WNE_P6']

UNITID
191649-2001        NaN
191649-2002        NaN
191649-2003    42400.0
191649-2004        NaN
191649-2005    42600.0
191649-2006        NaN
191649-2007    43500.0
191649-2008        NaN
191649-2009    40300.0
191649-2010        NaN
191649-2011    37700.0
191649-2012    36700.0
191649-2013    38300.0
191649-2014    39900.0
191649-2015        NaN
191649-2016        NaN
191649-2017        NaN
Name: MD_EARN_WNE_P6, dtype: float64

In [14]:
i = 1
df_cols[i * 10 : i * 10 + 10]

,category,type,friendly_name,explanation
LOCALE,school,integer,locale,Locale of institution
CCBASIC,school,integer,carnegie_basic,Carnegie Classification -- basic
CCUGPROF,school,integer,carnegie_undergrad,Carnegie Classification -- undergraduate profile
CCSIZSET,school,integer,carnegie_size_setting,Carnegie Classification -- size and setting
HBCU,school,integer,minority_serving.historically_black,Flag for Historically Black College and Univer...
MENONLY,school,integer,men_only,Flag for men-only college
WOMENONLY,school,integer,women_only,Flag for women-only college
RELAFFIL,school,integer,religious_affiliation,Religous affiliation of the institution
ADM_RATE,admissions,float,admission_rate.overall,Admission rate
SATVR25,admissions,float,sat_scores.25th_percentile.critical_reading,25th percentile of SAT scores at the instituti...


In [15]:
backfill = defaultdict(dict)
backfill_cols = [
    "LOCALE",
    "CCBASIC",
    "CCUGPROF",
    "CCSIZSET",
    "CURROPER",
    "NUM4_PUB",
    "NUM4_PRIV",
]
for row in df.query("YEAR==2017")[["INSTNM", *backfill_cols]].iterrows():
    for col in backfill_cols:
        backfill[row[1]["INSTNM"]][col] = row[1][col]

In [16]:
for col in backfill_cols:
    df.loc[:, col] = df.apply(lambda x: backfill[x["INSTNM"]].get(col), axis=1)

In [17]:
#cols_drop_pre = ["INSTNM"]
cols_drop_pre = []
categorical_keys = [
    "STABBR",
    "ACCREDAGENCY",
    "PREDDEG",
    "HIGHDEG",
    "CONTROL",
    "REGION",
    "LOCALE",
    "CCBASIC",
    "CCUGPROF",
    "CCSIZSET",
    "RELAFFIL",
   # "ICLEVEL",
]
cols_drop_post = ["CURROPER", "COSTT4_A", "COSTT4_P", "NUM4_PUB", "NUM4_PRIV", "NUM4"]

on_hold = [
#    "TUITIONFEE_IN",
#    "TUITIONFEE_OUT",
#    "TUITIONFEE_PROG",
    "MN_EARN_WNE_P6",
    "COUNT_NWNE_P6",
#    "COUNT_WNE_P6"
    "ICLEVEL",
]

In [18]:
sum(pd.notna(df['COUNT_WNE_P6']))

48735

In [19]:
# drop columns deemed unneeded
X = df.drop(cols_drop_pre, axis=1)
# must have at least 50 undergrads
X = X.query("UGDS >= 50")
X = X.query("CURROPER == 1")
X = X.query("PREDDEG != 4")
X = X.query("HIGHDEG not in [0,1]")
X["COSTT4_A"].fillna(value=X["COSTT4_P"], inplace=True)
X["COSTT4_P"].fillna(value=X["COSTT4_A"], inplace=True)
X.loc[:, "COSTT4"] = X.loc[:, "COSTT4_P"]
X["NUM4_PRIV"].fillna(value=X["NUM4_PUB"], inplace=True)
X["NUM4_PUB"].fillna(value=X["NUM4_PRIV"], inplace=True)
X.loc[:, "NUM4"] = X.loc[:, "NUM4_PUB"]
X = X.query("NUM4 > 250")
X = X.drop(cols_drop_post, axis=1)
X = X.drop(on_hold, axis=1)
print(X.YEAR.value_counts())

2017    1583
2016    1580
2015    1579
2014    1573
2013    1571
2012    1564
2011    1561
2009    1555
2010    1554
2008    1551
2007    1551
2006    1549
2004    1540
2005    1539
2003    1537
2002    1532
2001    1530
Name: YEAR, dtype: int64


In [20]:
regions = [v.split("(")[0].rstrip() for v in label_dict["REGION"].values()]

In [21]:
X["REGION"] = X.REGION.apply(lambda x: label_dict["REGION"][x].split("(")[0])

In [22]:
X["CONTROL"] = X.CONTROL.apply(lambda x: label_dict["CONTROL"][x])

In [23]:
X["MAIN"] = X.MAIN.apply(lambda x: label_dict["MAIN"][x])

In [24]:
X["HIGHDEG"] = X.HIGHDEG.apply(lambda x: label_dict["HIGHDEG"][x].split(" ")[0])

In [25]:
X["PREDDEG"] = X.PREDDEG.apply(
    lambda x: ["not_classified", "certificate", "associate", "bachelor", "grad"][x]
)

In [26]:
X.columns

Index(['INSTNM', 'STABBR', 'ACCREDAGENCY', 'HCM2', 'MAIN', 'NUMBRANCH',
       'PREDDEG', 'HIGHDEG', 'CONTROL', 'REGION',
       ...
       'COMPL_RPY_1YR_RT', 'COMPL_RPY_3YR_RT', 'COMPL_RPY_5YR_RT',
       'COMPL_RPY_7YR_RT', 'GRAD_DEBT_MDN', 'COUNT_WNE_P6', 'MD_EARN_WNE_P6',
       'PRGMOFR', 'YEAR', 'COSTT4'],
      dtype='object', length=101)

In [27]:
from pandas_summary import DataFrameSummary

In [28]:
df_sum = DataFrameSummary(X)
df_sum.columns_stats

,INSTNM,STABBR,ACCREDAGENCY,HCM2,MAIN,NUMBRANCH,PREDDEG,HIGHDEG,CONTROL,REGION,...,COMPL_RPY_1YR_RT,COMPL_RPY_3YR_RT,COMPL_RPY_5YR_RT,COMPL_RPY_7YR_RT,GRAD_DEBT_MDN,COUNT_WNE_P6,MD_EARN_WNE_P6,PRGMOFR,YEAR,COSTT4
counts,26449,26449,1583,1583,26449,26449,26449,26449,26449,26449,...,11251,11268,8262,5386,22340,12295,12330,370,26449,14114
uniques,1571,57,17,2,2,22,3,3,3,9,...,9385,9376,6965,4669,8050,4168,556,32,17,11618
missing,0,0,24866,24866,0,0,0,0,0,0,...,15198,15181,18187,21063,4109,14154,14119,26079,0,12335
missing_perc,0%,0%,94.01%,94.01%,0%,0%,0%,0%,0%,0%,...,57.46%,57.40%,68.76%,79.64%,15.54%,53.51%,53.38%,98.60%,0%,46.64%
types,categorical,categorical,categorical,bool,bool,numeric,categorical,categorical,categorical,categorical,...,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric


In [29]:
temp = df_sum.columns_stats.loc["missing_perc"].sort_values(ascending=False)
temp = temp.apply(lambda x: float(x[:-1]) / 100)
temp = pd.Series(
    [v for k, v in temp.items() if v > 0.5 and k != "MD_EARN_WNE_P6"],
    index=[k for k, v in temp.items() if v > 0.5 and k != "MD_EARN_WNE_P6"],
)
temp

RELAFFIL            0.9886
PRGMOFR             0.9860
TUITIONFEE_PROG     0.9860
ACTWR75             0.9480
ACTWR25             0.9480
ACTWRMID            0.9480
ACCREDAGENCY        0.9401
WOMENONLY           0.9401
MENONLY             0.9401
HBCU                0.9401
HCM2                0.9401
SATWR25             0.8364
SATWR75             0.8364
SATWRMID            0.8364
COMPL_RPY_7YR_RT    0.7964
COMPL_RPY_5YR_RT    0.6876
ACTMTMID            0.5915
ACTMT75             0.5915
ACTMT25             0.5914
ACTEN75             0.5910
ACTEN25             0.5910
ACTENMID            0.5910
DISTANCEONLY        0.5837
COMPL_RPY_1YR_RT    0.5746
COMPL_RPY_3YR_RT    0.5740
COUNT_WNE_P6        0.5351
ACTCM75             0.5128
ACTCM25             0.5128
ACTCMMID            0.5128
SATVRMID            0.5018
SATVR25             0.5017
SATVR75             0.5017
dtype: float64

In [30]:
X = X.drop(columns=temp[temp > 0.5].index, axis=1)
print(f"dropped {len(temp[temp>0.75].index)} columns")

dropped 15 columns


In [31]:
X.columns

Index(['INSTNM', 'STABBR', 'MAIN', 'NUMBRANCH', 'PREDDEG', 'HIGHDEG',
       'CONTROL', 'REGION', 'LOCALE', 'CCBASIC', 'CCUGPROF', 'CCSIZSET',
       'ADM_RATE', 'SATMT25', 'SATMT75', 'SATMTMID', 'PCIP01', 'PCIP03',
       'PCIP04', 'PCIP05', 'PCIP09', 'PCIP10', 'PCIP11', 'PCIP12', 'PCIP13',
       'PCIP14', 'PCIP15', 'PCIP16', 'PCIP19', 'PCIP22', 'PCIP23', 'PCIP24',
       'PCIP25', 'PCIP26', 'PCIP27', 'PCIP29', 'PCIP30', 'PCIP31', 'PCIP38',
       'PCIP39', 'PCIP40', 'PCIP41', 'PCIP42', 'PCIP43', 'PCIP44', 'PCIP45',
       'PCIP46', 'PCIP47', 'PCIP48', 'PCIP49', 'PCIP50', 'PCIP51', 'PCIP52',
       'PCIP54', 'UGDS', 'PPTUG_EF', 'TUITIONFEE_IN', 'TUITIONFEE_OUT',
       'INEXPFTE', 'AVGFACSAL', 'PFTFAC', 'PCTPELL', 'C150_4', 'PCTFLOAN',
       'UG25ABV', 'GRAD_DEBT_MDN', 'MD_EARN_WNE_P6', 'YEAR', 'COSTT4'],
      dtype='object')

In [32]:
with open("raw_df.pickle", "wb") as f:
    pickle.dump(X, f)

## See fixing earnings data notebook for intermediate steps.

#### Load new X from 'merged_earnings_df'

In [33]:
X = data = pickle.load(open('merged_earnings_df.pickle', 'rb'))

In [34]:
X

,INSTNM,STABBR,ACCREDAGENCY,HCM2,MAIN,NUMBRANCH,PREDDEG,HIGHDEG,CONTROL,REGION,...,PRGMOFR,YEAR,COSTT4,COUNT_WNE_P6_year_1,MD_EARN_WNE_P6_year_1,_merged_1,COUNT_WNE_P6_year_2,MD_EARN_WNE_P6_year_2,_merged_2,MD_EARN_AVG
0,Alabama A & M University,AL,NaN,NaN,Main campus,1.0,bachelor,Graduate,Public,Southeast,...,NaN,2002.0,NaN,1291.0,25000.0,both,0.0,0.0,left_only,25000.0
1,University of Alabama at Birmingham,AL,NaN,NaN,Main campus,1.0,bachelor,Graduate,Public,Southeast,...,NaN,2002.0,NaN,2882.0,34700.0,both,0.0,0.0,left_only,34700.0
2,University of Alabama in Huntsville,AL,NaN,NaN,Main campus,1.0,bachelor,Graduate,Public,Southeast,...,NaN,2002.0,NaN,1082.0,38700.0,both,0.0,0.0,left_only,38700.0
3,Alabama State University,AL,NaN,NaN,Main campus,1.0,bachelor,Graduate,Public,Southeast,...,NaN,2002.0,NaN,1520.0,20600.0,both,0.0,0.0,left_only,20600.0
4,The University of Alabama,AL,NaN,NaN,Main campus,1.0,bachelor,Graduate,Public,Southeast,...,NaN,2002.0,NaN,2917.0,35400.0,both,0.0,0.0,left_only,35400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11333,Pierpont Community and Technical College,WV,NaN,NaN,Main campus,1.0,associate,Associate,Public,Southeast,...,NaN,2008.0,NaN,0.0,0.0,left_only,0.0,25300.0,both,25300.0
11334,University of California-Merced,CA,NaN,NaN,Main campus,1.0,bachelor,Graduate,Public,Far West,...,NaN,2008.0,NaN,0.0,0.0,left_only,1189.0,36900.0,both,36900.0
11335,Pima Medical Institute-Las Vegas,NV,NaN,NaN,Not main campus,9.0,certificate,Associate,Private for-profit,Far West,...,9.0,2008.0,NaN,0.0,0.0,left_only,8268.0,26700.0,both,26700.0
11336,Johnson & Wales University-Charlotte,NC,NaN,NaN,Not main campus,4.0,associate,Bachelor's,Private nonprofit,Southeast,...,NaN,2008.0,NaN,0.0,0.0,left_only,7340.0,31900.0,both,31900.0


# One hot encoding begins here

In [35]:
X = pd.get_dummies(
    X,
    dummy_na=True,
    columns=[*[col for col in categorical_keys if col in X.columns], "STABBR"],
    drop_first=True,
)

In [36]:
X.loc[:, "MD_EARN_AVG"].isna().value_counts()

False    11332
Name: MD_EARN_AVG, dtype: int64

In [37]:
y = X.MD_EARN_AVG
X.drop("MD_EARN_AVG", axis=1, inplace=True)

In [38]:
X.YEAR.value_counts()

2007.0    1677
2006.0    1676
2005.0    1666
2008.0    1586
2003.0    1579
2004.0    1575
2002.0    1573
Name: YEAR, dtype: int64

In [ ]:
with open("clean_data.pickle", "wb") as f:
    pickle.dump(
        {
            "X": X,
            'y': y
        },
        f,
    )

In [44]:
df_cols.loc['C150_4']

category                                                completion
type                                                         float
friendly_name                            completion_rate_4yr_150nt
explanation      Completion rate for first-time, full-time stud...
Name: C150_4, dtype: object